## Bibliotecas

In [3]:
# Célula 1: Imports básicos
import os
import sys
import requests
from pathlib import Path
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

print(" Notebook de Download de Dados")
print(" Usando DuckDB para manipulação eficiente")
print("="*60)

 Notebook de Download de Dados
 Usando DuckDB para manipulação eficiente


## Criando estruturas de pastas

In [6]:
# Criar estrutura de diretórios
BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data"
DATA_RAW = DATA_DIR / "raw"
DATA_RAW_SIM = DATA_RAW / "sim"
DATA_RAW_ATLAS = DATA_RAW / "atlas"
DUCKDB_DIR = BASE_DIR / "duckdb"

# Criar todos os diretórios
DATA_RAW_SIM.mkdir(parents=True, exist_ok=True)
DATA_RAW_ATLAS.mkdir(parents=True, exist_ok=True)
DUCKDB_DIR.mkdir(parents=True, exist_ok=True)

print(" Estrutura de pastas criada:")
print(f"    {DATA_RAW_SIM}")
print(f"    {DATA_RAW_ATLAS}")
print(f"    {DUCKDB_DIR}")

 Estrutura de pastas criada:
    ../data/raw/sim
    ../data/raw/atlas
    ../duckdb


## Instalar dependências

In [7]:
#  Instalar dependências necessárias
def install_package(package):
    """Instala pacote se não estiver disponível"""
    try:
        __import__(package.split('[')[0].split('=')[0])
        print(f"  ✓ {package}")
        return True
    except ImportError:
        print(f"  ---  Instalando {package}...")
        os.system(f"{sys.executable} -m pip install -q {package}")
        print(f"  V {package} - instalado!")
        return True

print(" Verificando dependências...\n")

packages = [
    'duckdb',
    'pysus',
    'pyarrow',
    'pandas',
    'tqdm',
    'requests'
]

for pkg in packages:
    install_package(pkg)

print("\n Todas as dependências instaladas!")

 Verificando dependências...

  ---  Instalando duckdb...
  V duckdb - instalado!
  ---  Instalando pysus...


  V pysus - instalado!
  ✓ pyarrow
  ✓ pandas
  ✓ tqdm
  ✓ requests

 Todas as dependências instaladas!


## Importar bibliotecas

In [8]:
# Importar bibliotecas
import duckdb
import pandas as pd
from pysus.online_data import SIM

print(" Bibliotecas importadas com sucesso:")
print("    DuckDB")
print("    PySUS (SIM)")
print("    Pandas")

 Bibliotecas importadas com sucesso:
    DuckDB
    PySUS (SIM)
    Pandas


## Conectar ao dunckDB

In [10]:
# Criar/Conectar ao banco DuckDB
DB_PATH = DUCKDB_DIR / "mortalidade.duckdb"

# Criar conexão
conn = duckdb.connect(str(DB_PATH))

print(f" Conexão DuckDB estabelecida!")
print(f" Banco: {DB_PATH}")

# Verificar versão
version = conn.execute("SELECT version()").fetchone()[0]
print(f" Versão: {version}")

 Conexão DuckDB estabelecida!
 Banco: ../duckdb/mortalidade.duckdb
 Versão: v1.4.1


In [17]:
# Célula 6: Teste - Baixar apenas 2010 de um estado
print("="*60)
print("🧪 TESTE: Baixando dados de 2010 (SP apenas)")
print("="*60)

try:
    print("📥 Baixando dados de SP em 2010...", end=" ")
    
    from pysus.preprocessing.decoders import decode_SIM
    
    # Baixar apenas SP como teste
    files = SIM.download(
        groups='CID10',
        states='SP',         # Um estado específico
        years=2010
    )
    
    print(f"✅ Download OK!")
    print(f"📁 Arquivos: {files}")
    
    # Decodificar arquivo
    print(f"\n🔄 Lendo arquivo...")
    df_test = decode_SIM(files[0])
    
    if df_test is not None and len(df_test) > 0:
        print(f"✅ Leitura bem-sucedida!")
        print(f"📊 Registros: {len(df_test):,}")
        print(f"📊 Colunas: {len(df_test.columns)}")
        
        print(f"\n📋 Primeiras 10 colunas:")
        for i, col in enumerate(df_test.columns[:10], 1):
            print(f"   {i:2d}. {col}")
        
        print(f"\n📋 Preview (3 primeiras linhas):")
        print(df_test.head(3))
        
        print(f"\n✅ TESTE BEM-SUCEDIDO! Pode executar a próxima célula.")
    
except Exception as e:
    print(f"❌ Erro: {e}")
    import traceback
    traceback.print_exc()

🧪 TESTE: Baixando dados de 2010 (SP apenas)
📥 Baixando dados de SP em 2010... ❌ Erro: cannot import name 'decode_SIM' from 'pysus.preprocessing.decoders' (/home/pedroconrado/mortalidade-cardiovascular-ml/venv/lib/python3.12/site-packages/pysus/preprocessing/decoders.py)


Traceback (most recent call last):
  File "/tmp/ipykernel_4843/1577240817.py", line 9, in <module>
    from pysus.preprocessing.decoders import decode_SIM
ImportError: cannot import name 'decode_SIM' from 'pysus.preprocessing.decoders' (/home/pedroconrado/mortalidade-cardiovascular-ml/venv/lib/python3.12/site-packages/pysus/preprocessing/decoders.py)


In [14]:
# Célula EXTRA: Verificar PySUS
import pysus

print(f"📦 Versão do PySUS: {pysus.__version__}")

# Ver documentação da função
print("\n📋 Documentação do SIM.download:")
help(SIM.download)

📦 Versão do PySUS: 1.0.0

📋 Documentação do SIM.download:
Help on function download in module pysus.online_data.SIM:

download(groups: Union[str, list], states: Union[str, list], years: Union[str, list, int], data_dir: str = '/home/pedroconrado/pysus')
    Downloads data directly from Datasus ftp server
    :param groups: either CID9, CID10 or both
    :param states: two-letter state identifier: MG == Minas Gerais
                   can be a list
    :param years: years to download
    :return: a list of downloaded files

